<a href="https://colab.research.google.com/github/Prathamesh-Chavan-98/NLP-Next-Word-Predictor-using-LSTM/blob/main/_NLP_Next_Word_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import regex as re


In [ ]:
!pip install pypdf2

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import regex as re
import PyPDF2 # Import PyPDF2 to read PDF files

def file_to_sentence_list(file_path):
    with open(file_path, 'rb') as file: # Open in binary read mode 'rb'
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()

    # Splitting the text into sentences using
    # delimiters like '.', '?', and '!'
    sentences = [sentence.strip() for sentence in re.split(
        r'(?<=[.!?])\s+', text) if sentence.strip()]

    return sentences

file_path = '/content/MACHINE LEARNING(R17A0534) (1).pdf'
text_data = file_to_sentence_list(file_path)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1

# Create input sequences
input_sequences = []
for line in text_data:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

# Pad sequences and split into predictors and label
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(
	input_sequences, maxlen=max_sequence_len, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]

# Convert target data to one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=total_words)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.2 MB/s eta 0:00:00


In [ ]:
print(X.shape)

(38542, 318)


In [ ]:
print(y.shape)

(38542, 4169)


In [ ]:
# Define the model
model = Sequential()
model.add(Embedding(total_words, 10,
					input_length=max_sequence_len-1))
model.add(LSTM(128))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy',
			optimizer='adam', metrics=['accuracy'])


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 318, 10)           41690     
                                                                 
 lstm (LSTM)                 (None, 128)               71168     
                                                                 
 dense (Dense)               (None, 4169)              537801    
                                                                 
Total params: 650659 (2.48 MB)
Trainable params: 650659 (2.48 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Train the model
model.fit(X, y, epochs=100, verbose=1)


Epoch 1/100
1205/1205 [==============================] - 19s 16ms/step - loss: 4.2846 - accuracy: 0.1974
Epoch 2/100
1205/1205 [==============================] - 19s 15ms/step - loss: 4.1115 - accuracy: 0.2164
Epoch 3/100
1205/1205 [==============================] - 18s 15ms/step - loss: 3.9471 - accuracy: 0.2344
Epoch 4/100
1205/1205 [==============================] - 20s 17ms/step - loss: 3.7920 - accuracy: 0.2544
Epoch 5/100
1205/1205 [==============================] - 18s 15ms/step - loss: 3.6427 - accuracy: 0.2760
Epoch 6/100
1205/1205 [==============================] - 19s 16ms/step - loss: 3.5055 - accuracy: 0.2956
Epoch 7/100
1205/1205 [==============================] - 18s 15ms/step - loss: 3.3769 - accuracy: 0.3167
Epoch 8/100
1205/1205 [==============================] - 18s 15ms/step - loss: 3.2554 - accuracy: 0.3349
Epoch 9/100
1205/1205 [==============================] - 18s 15ms/step - loss: 3.1434 - accuracy: 0.3515
Epoch 10/100
1205/1205 [==============================]

In [ ]:
# Generate next word predictions
seed_text = "linear regression is a "
next_words = 5

for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences(
		[token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted_probs = model.predict(token_list)
	predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
	seed_text += " " + predicted_word

print("Next predicted words:", seed_text)


1/1 [==============================] - 0s 19ms/step
Next predicted words: linear regression is a  linear example for the dependent
